# Week 05 Classification Evaluation Assignment

- Course: Z604 Music Data Mining 
- Instructor: Kahyun Choi 

This week's assignment uses the same classifier for the Week 05 assignment. 

In [1]:
# we will be reusing dataset and utilities from W04
import os
if os.path.exists('W04'): 
    !rm -fr 'W04/'
!git clone https://github.com/music-data-mining/W04.git
%cd W04

Cloning into 'W04'...
remote: Enumerating objects: 245, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 245 (delta 6), reused 10 (delta 4), pack-reused 233
Receiving objects: 100% (245/245), 42.97 MiB | 28.70 MiB/s, done.
Resolving deltas: 100% (7/7), done.
/content/W04


# Setup

In [2]:
# Common imports
import os
import numpy as np
import pandas as pd
from scipy import stats

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "training_linear_models"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "slide_images", fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

### Part of the Assignment Week 03/04 

### Q1-4. Softmax Regression using 1) means and vars of segment timbre, chroma, and loudmax and 2)means and vars of beats timbre, chroma, and loudmax

* Build a softmax regression classifier to solve the music mood classification problem. The four mood classes are four quadrants of Russell’s emotion circumplex.
 
* Use 10% of the dataset for the test set. Report classification scores of the training set and the test set. 


* Use the following features:
seg_timbre_mean, seg_timbre_var, seg_chroma_mean, seg_chroma_var, seg_loudmax_mean, seg_loudmax_var, bt_timbre_mean, bt_timbre_var, bt_chroma_mean, bt_chroma_var, bt_loudmax_mean, bt_loudmax_var

In [3]:
data = pd.read_csv('moodmsdfeatures.csv')  # load data set

In [4]:
import csv
import hdf5_getters
import beat_aligned_feats as baf

songreader = csv.DictReader(open('moodmsdfeatures.csv'))
# init segment features containers
seg_timbre_mean = np.empty((0,12), dtype=float)
seg_timbre_var = np.empty((0,12), dtype=float)
seg_chroma_mean = np.empty((0,12), dtype=float)
seg_chroma_var = np.empty((0,12), dtype=float)
seg_loudmax_mean = np.empty((0,1), dtype=float)
seg_loudmax_var = np.empty((0,1), dtype=float)
# init beat features containers
bt_timbre_mean = np.empty((0,12), dtype=float)
bt_timbre_var = np.empty((0,12), dtype=float)
bt_chroma_mean = np.empty((0,12), dtype=float)
bt_chroma_var = np.empty((0,12), dtype=float)
bt_loudmax_mean = np.empty((0,1), dtype=float)
bt_loudmax_var = np.empty((0,1), dtype=float)

for song in songreader:
    trid = song['MSD_track_id']
    filename = 'deezer_MSD/' + trid + '.h5'
    h5 = hdf5_getters.open_h5_file_read(filename)
    # extract segment features
    seg_timbre = hdf5_getters.get_segments_timbre(h5)
    seg_timbre_mean = np.vstack((seg_timbre_mean, np.mean(seg_timbre, axis = 0)))
    seg_timbre_var = np.vstack((seg_timbre_var, np.var(seg_timbre, axis = 0)))

    seg_chroma = hdf5_getters.get_segments_pitches(h5)
    seg_chroma_mean = np.vstack((seg_chroma_mean, np.mean(seg_chroma, axis = 0)))
    seg_chroma_var = np.vstack((seg_chroma_var, np.var(seg_chroma, axis = 0)))
    
    seg_loudmax = hdf5_getters.get_segments_loudness_max(h5)
    seg_loudmax_mean = np.vstack((seg_loudmax_mean, np.mean(seg_loudmax, axis = 0)))
    seg_loudmax_var = np.vstack((seg_loudmax_var, np.var(seg_loudmax, axis = 0)))    
    # extract beat features
    bt_timbre = baf.get_bttimbre(filename).T
    bt_timbre_mean = np.vstack((bt_timbre_mean, np.mean(bt_timbre, axis = 0)))
    bt_timbre_var = np.vstack((bt_timbre_var, np.var(bt_timbre, axis = 0)))
    
    bt_chroma = baf.get_btchromas(filename).T
    bt_chroma_mean = np.vstack((bt_chroma_mean, np.mean(bt_chroma, axis = 0)))
    bt_chroma_var = np.vstack((bt_chroma_var, np.var(bt_chroma, axis = 0)))    
    
    bt_loudmax = baf.get_btloudnessmax(filename)
    bt_loudmax = np.transpose(bt_loudmax)
    bt_loudmax_mean = np.vstack((bt_loudmax_mean, np.mean(bt_loudmax, axis = 0)))
    bt_loudmax_var = np.vstack((bt_loudmax_var, np.var(bt_loudmax, axis = 0)))  

In [30]:
from sklearn import preprocessing
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split

X = np.hstack((seg_timbre_mean,
               seg_timbre_var,
               seg_chroma_mean,
               seg_chroma_var,
               seg_loudmax_mean,
               seg_loudmax_var,
               bt_timbre_mean,
               bt_timbre_var,
               bt_chroma_mean,
               bt_chroma_var,
               bt_loudmax_mean,
               bt_loudmax_var))
y = data['quadrant'].values.reshape(-1,)

# X = preprocessing.scale(X)  # try this line to see how a simple scaler improve the performance
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

reg = SGDClassifier(loss="log", max_iter = 1e9)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

score = reg.score(X_train, y_train)
print("Train accuracy score: %.4f" % score)
score = reg.score(X_test, y_test)
print("Test accuracy score: %.4f" % score)

Train accuracy score: 0.7164
Test accuracy score: 0.5217


## Assignment Week 05

#Q1. Report evaluation results of the classifier that you built for the Week 04 assignment's Q1-4

Report the scores of the following evaluation metrics. (1pt)
* Normalized confusion matrix
* Accuracy
* Precision
* Recall
* F1



Pick the most confusing pair of classes *by visualizing the normalized confusion matrix.* <--- Please pick the most confusing pair of classes. (1pt)

# Q2. Explore MIREX evaluation metrics

MIREX has wiki pages for individual MIR tasks since 2005. 

First, visit the main page (https://www.music-ir.org/mirex/wiki/2019:Main_Page) and pick a MIREX main page of a particular year, and then click the task name of your interest. 

On the task wiki, you can find detailed information about the task, including the task description, data, and evaluation metrics. 

Pick one evaluation metric used by a task of your interest in a particular year and briefly introduce it here. (1pt)

Task:

Year: 

Evaluation metric:


